In [1]:
import cv2 as cv
import tensorflow as tf
import numpy as np
from keras.models import model_from_json
import matplotlib.pyplot as plt
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import *
from tensorflow.keras import layers , models, optimizers
from tensorflow.keras.models import Sequential, Model
import math
from mtcnn import MTCNN

In [16]:
def boxing(img, emotion_count, detector):
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    lis=detector.detect_faces(img)
    for i in lis:
        x,y,w,h=i["box"]
        gray_face = gray[y:y + h, x:x + w] 
        resized=cv.resize(gray_face, (48,48))
        image_tensor = tf.convert_to_tensor(resized, dtype=tf.float32)
        # Add dimension to match with input mode 
        image_tensor = tf.expand_dims(image_tensor, 2)
        image_tensor = tf.expand_dims(image_tensor, 0)
        image_tensor=image_tensor/255
        res=emotion_model.predict(image_tensor).argmax()
        emotion_count[res]+=1

In [3]:
emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}

# create model structure
emotion_model = Sequential()

#emotion_model.add(data_augmentation)

emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
emotion_model.add(BatchNormalization())

emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(BatchNormalization())

emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(BatchNormalization())

emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(BatchNormalization())
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(BatchNormalization())

emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(BatchNormalization())

emotion_model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
emotion_model.add(BatchNormalization())
emotion_model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
emotion_model.add(BatchNormalization())

emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(BatchNormalization())

emotion_model.add(Flatten())
emotion_model.add(Dense(256, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(BatchNormalization())

emotion_model.add(Dense(128, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(BatchNormalization())

emotion_model.add(Dense(7, activation='softmax'))

emotion_model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])

# load weights into model
emotion_model.load_weights("models/emotion_model.h5")

In [14]:
def emotion_detection(video_path):
    emotion_count=[0,0,0,0,0,0,0]
    detector = MTCNN()
    video = cv.VideoCapture(video_path)
    fps = video.get(cv.CAP_PROP_FPS)
    frame_count = int(video.get(cv.CAP_PROP_FRAME_COUNT))
    duration = math.floor(frame_count/fps)
    print(duration)
    for i in range(0,duration):
        frame_id = int(fps*(i))
        video.set(cv.CAP_PROP_POS_FRAMES, frame_id)
        ret, frame = video.read()
        if ret==True:
            boxing(frame, emotion_count, detector)
    return emotion_count

In [5]:
emotion_count=emotion_detection('sample.mp4')
print(emotion_count)

1/1 [==============================] - 0s 45ms/step


1/1 [==============================] - 0s 56ms/step
[0, 0, 0, 33, 2, 3, 0]


In [15]:
emotion_count=emotion_detection('sample2.mp4')
print(emotion_count)

15
1/1 [==============================] - 0s 318ms/step
[{'box': [1332, 353, 1127, 1396], 'confidence': 0.9998393058776855, 'keypoints': {'left_eye': (1707, 894), 'right_eye': (2201, 829), 'nose': (2044, 1064), 'mouth_left': (1846, 1432), 'mouth_right': (2251, 1380)}}]
1/1 [==============================] - 0s 54ms/step
[{'box': [1301, 339, 1085, 1365], 'confidence': 0.9995575547218323, 'keypoints': {'left_eye': (1672, 881), 'right_eye': (2155, 804), 'nose': (2002, 1087), 'mouth_left': (1814, 1419), 'mouth_right': (2203, 1365)}}]
1/1 [==============================] - 0s 51ms/step
[{'box': [1306, 378, 1089, 1382], 'confidence': 0.9996933937072754, 'keypoints': {'left_eye': (1683, 932), 'right_eye': (2166, 853), 'nose': (2020, 1147), 'mouth_left': (1823, 1472), 'mouth_right': (2214, 1413)}}]
1/1 [==============================] - 0s 48ms/step
[{'box': [1320, 369, 1120, 1434], 'confidence': 0.9993220567703247, 'keypoints': {'left_eye': (1719, 945), 'right_eye': (2208, 864), 'nose': (2069

1/1 [==============================] - 0s 50ms/step
[{'box': [1269, 340, 1117, 1410], 'confidence': 0.9995726943016052, 'keypoints': {'left_eye': (1642, 920), 'right_eye': (2123, 832), 'nose': (1971, 1127), 'mouth_left': (1795, 1458), 'mouth_right': (2185, 1393)}}]
1/1 [==============================] - 0s 52ms/step
[{'box': [1262, 342, 1144, 1421], 'confidence': 0.9981080293655396, 'keypoints': {'left_eye': (1641, 927), 'right_eye': (2135, 812), 'nose': (1998, 1142), 'mouth_left': (1830, 1466), 'mouth_right': (2216, 1379)}}]
1/1 [==============================] - 0s 54ms/step
[]
1/1 [==============================] - 0s 52ms/step
[]
1/1 [==============================] - 0s 55ms/step
[{'box': [1446, 428, 1131, 1473], 'confidence': 0.9986156225204468, 'keypoints': {'left_eye': (1825, 1006), 'right_eye': (2338, 985), 'nose': (2131, 1346), 'mouth_left': (1885, 1587), 'mouth_right': (2303, 1578)}}]
1/1 [==============================] - 0s 62ms/step
[{'box': [1564, 395, 1131, 1476], 'conf

In [10]:
emotion_count=emotion_detection('sample3.mp4')
print(emotion_count)

22
1/1 [==============================] - 0s 255ms/step


1/1 [==============================] - 0s 91ms/step


1/1 [==============================] - 0s 60ms/step
[0, 0, 0, 0, 2, 20, 0]
